In [4]:
# load a coffea file
from coffea.util import load
import os
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib
matplotlib.rcParams["figure.dpi"] = 300
import matplotlib as mpl
import boost_histogram as bh

import vector
vector.register_awkward


<function vector.register_awkward() -> 'None'>

In [2]:
fpath = "/work/tharte/datasets/btag_efficiency_outputs/btag_eff_nobcuts_all_kl_proper_skimming/output_all.coffea"
outfile = load(fpath)

In [22]:
outfile["columns"]
cols = outfile["columns"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE"]["inclusive"]
print(cols.keys())

dict_keys(['weight', 'JetGood_N', 'JetGood_pt', 'JetGood_eta', 'JetGood_phi', 'JetGood_mass', 'JetGood_btagPNetB_5wp', 'JetGood_btagPNetB_3wp', 'JetGood_btagPNetB'])


In [43]:
colpt = cols["JetGood_pt"].value
coleta = cols["JetGood_eta"].value
colb = cols["JetGood_btagPNetB"].value
print(colpt)
target = {"pt": 14.7, "eta": 2.5, "btag": 0.744}
tol = {"pt": 0.1, "eta": 0.1, "btag": 0.001}
masks = []
for col, var in zip([colpt,coleta,colb],["pt","eta","btag"]):
    masks.append((col > target[var] - tol[var]) & (col < target[var] + tol[var]))
idx = ak.where(masks[0] & masks[1] & masks[2])
print(idx)
event =0
jetsum = 0
for i in cols["JetGood_N"].value:
    jetsum+=i
    if jetsum>1010328:
        print(f"Found event number {event}")
        break
    event+=1

[148.17822   46.262566 137.4233   ...  69.299     78.1893   113.38634 ]
(<Array [1010328] type='1 * int64'>,)
Found event number 199057


In [44]:
for key, value in cols.items():
    if "JetGood" in key and key != "JetGood_N":
        print(f"Single jet {key}: {value.value[1010328]}")
        print(f"Full event {key}: {ak.unflatten(value.value, cols['JetGood_N'].value)[199057]}")
    else:
        print(f"{key}: {value.value[199057]}")

weight: 69.35518548233219
JetGood_N: 8
Single jet JetGood_pt: 14.743119239807129
Full event JetGood_pt: [124, 14.7, 115, 79.4, 127, 91.5, 43.1, 36.5]
Single jet JetGood_eta: 2.49609375
Full event JetGood_eta: [0.802, 2.5, 0.123, 0.843, 0.13, 1.8, -1.97, -0.759]
Single jet JetGood_phi: -0.2662353515625
Full event JetGood_phi: [2.52, -0.266, -2.47, 1.57, 0.53, -1.9, -0.51, -1.59]
Single jet JetGood_mass: 3.0204966068267822
Full event JetGood_mass: [23.3, 3.02, 11.9, 10.8, 12.9, 15.4, 8.71, 5.89]
Single jet JetGood_btagPNetB_5wp: 2
Full event JetGood_btagPNetB_5wp: [4, 2, 1, -1, -1, -1, -1, -1]
Single jet JetGood_btagPNetB_3wp: 2
Full event JetGood_btagPNetB_3wp: [2, 2, 1, -1, -1, -1, -1, -1]
Single jet JetGood_btagPNetB: 0.74365234375
Full event JetGood_btagPNetB: [1, 0.744, 0.586, 0.0384, 0.0041, 0.00576, 0.00394, 0.00938]


In [58]:
import correctionlib

cset = correctionlib.CorrectionSet.from_file("/work/tharte/datasets/AnalysisConfigs/configs/HH4b_common/params/btag_efficiencies_multipleWP/btag_efficiencies_btagPNetB_2022_postEE.json")
cset_klm2_XT = cset["HH4b_kl-m2p00_kt-1p00_wp_XT"]
cset_klm2_T = cset["HH4b_kl-m2p00_kt-1p00_wp_T"]
cset_klm2_M = cset["HH4b_kl-m2p00_kt-1p00_wp_M"]

In [60]:
print(cset_klm2_XT.evaluate(14.743119239807129, 2.49609375, float(4)))
print(cset_klm2_T.evaluate(14.743119239807129, 2.49609375, float(4)))
print(cset_klm2_M.evaluate(14.743119239807129, 2.49609375, float(4)))

0.01408450704225352
0.01408450704225352
0.11267605633802816


In [122]:
import correctionlib

cset = correctionlib.CorrectionSet.from_file("/work/tharte/datasets/AnalysisConfigs/configs/HH4b_btagging/btag_eff_nobcuts_all_kl_proper_skimming_load_hist_last_only_m2p00/eff_all_kl_proper_skiming_bugfixed/btag_efficiencies_btagPNetB_2022_postEE.json")
cset_klm2_XT = cset["HH4b_kl-m2p00_kt-1p00_wp_XT"]
cset_klm2_T = cset["HH4b_kl-m2p00_kt-1p00_wp_T"]
cset_klm2_M = cset["HH4b_kl-m2p00_kt-1p00_wp_M"]

In [123]:
print(cset_klm2_XT.evaluate(14.743119239807129, 2.49609375, float(4)))
print(cset_klm2_T.evaluate(14.743119239807129, 2.49609375, float(4)))
print(cset_klm2_M.evaluate(14.743119239807129, 2.49609375, float(4)))

0.01408450704225352
0.01408450704225352
0.11267605633802816


In [65]:
cset_sf = correctionlib.CorrectionSet.from_file("/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/POG/BTV/2022_Summer22EE/btagging.json.gz")

In [68]:
cset_sf["particleNet_wp_values"].evaluate("XT")

0.8033

In [179]:


hist_T = outfile["variables"]["bjets_particleNet_T_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE"]
print(hist_T)
# print(hist_T[{"BJetGood_particleNet_T.hadronFlavour": 1, "BJetGood_particleNet_T.abseta": 1, "BJetGood_particleNet_T.pt": -1}])
print(hist_T[{"BJetGood_particleNet_T.pt": -1}].values())

Hist(
  StrCategory(['inclusive'], name='cat', label='Category'),
  StrCategory(['nominal', 'pileupDown', 'pileupUp'], name='variation', label='Variation'),
  Variable([20, 30, 50, 70, 100, 140, 200, 300, 600, 1000], name='BJetGood_particleNet_T.pt', label='BJetGood_particleNet_T pT'),
  Variable([0, 2.4, 2.5], name='BJetGood_particleNet_T.abseta', label='BJetGood_particleNet_T eta'),
  Variable([-0.5, 3.5, 4.5, 5.5], name='BJetGood_particleNet_T.hadronFlavour', label='BJetGood_particleNet_T flav'),
  storage=Weight()) # Sum: WeightedSum(value=2.50308, variance=8.3877e-06) (WeightedSum(value=2.50367, variance=8.38967e-06) with flow)
[[[[0.         0.         0.00108236]
   [0.         0.         0.        ]]

  [[0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.         0.         0.        ]
   [0.         0.         0.        ]]]]


### Compare Hists of all samples and single sample

In [5]:
allfiles = "/work/tharte/datasets/btag_efficiency_outputs/btag_eff_nobcuts_all_kl_proper_skimming_load_hist_last/output_all.coffea"
m2p0file = "/work/tharte/datasets/btag_efficiency_outputs/btag_eff_nobcuts_all_kl_proper_skimming_load_hist_last_only_m2p00/output_all.coffea"

oall = load(allfiles)
om2 = load(m2p0file)

histfromall = oall["variables"]["bjets_particleNet_T_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
histfromm2 = om2["variables"]["bjets_particleNet_T_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']

In [115]:
print(histfromall.values)
print(histfromm2.values)

<bound method Histogram.values of Hist(
  StrCategory(['inclusive'], name='cat', label='Category'),
  StrCategory(['nominal', 'pileupDown', 'pileupUp'], name='variation', label='Variation'),
  Variable([20, 30, 50, 70, 100, 140, 200, 300, 600, 1000], name='BJetGood_particleNet_T.pt', label='BJetGood_particleNet_T pT'),
  Variable([0, 2.4, 2.5], name='BJetGood_particleNet_T.abseta', label='BJetGood_particleNet_T eta'),
  Variable([-0.5, 3.5, 4.5, 5.5], name='BJetGood_particleNet_T.hadronFlavour', label='BJetGood_particleNet_T flav'),
  storage=Weight()) # Sum: WeightedSum(value=2.50308, variance=8.3877e-06) (WeightedSum(value=2.50367, variance=8.38967e-06) with flow)>
<bound method Histogram.values of Hist(
  StrCategory(['inclusive'], name='cat', label='Category'),
  StrCategory(['nominal', 'pileupDown', 'pileupUp'], name='variation', label='Variation'),
  Variable([20, 30, 50, 70, 100, 140, 200, 300, 600, 1000], name='BJetGood_particleNet_T.pt', label='BJetGood_particleNet_T pT'),
  V

In [120]:
histfromall_alljets = oall["variables"]["jets_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
histfromm2_alljets = om2["variables"]["jets_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']

In [121]:
print(histfromall_alljets)
print(histfromm2_alljets)

Hist(
  StrCategory(['inclusive'], name='cat', label='Category'),
  StrCategory(['nominal', 'pileupDown', 'pileupUp'], name='variation', label='Variation'),
  Variable([20, 30, 50, 70, 100, 140, 200, 300, 600, 1000], name='JetGood.pt', label='JetGood pT'),
  Variable([0, 2.4, 2.5], name='JetGood.abseta', label='JetGood eta'),
  Variable([-0.5, 3.5, 4.5, 5.5], name='JetGood.hadronFlavour', label='JetGood flav'),
  storage=Weight()) # Sum: WeightedSum(value=4.89237, variance=1.63941e-05) (WeightedSum(value=4.93195, variance=1.65267e-05) with flow)
Hist(
  StrCategory(['inclusive'], name='cat', label='Category'),
  StrCategory(['nominal', 'pileupDown', 'pileupUp'], name='variation', label='Variation'),
  Variable([20, 30, 50, 70, 100, 140, 200, 300, 600, 1000], name='JetGood.pt', label='JetGood pT'),
  Variable([0, 2.4, 2.5], name='JetGood.abseta', label='JetGood eta'),
  Variable([-0.5, 3.5, 4.5, 5.5], name='JetGood.hadronFlavour', label='JetGood flav'),
  storage=Weight()) # Sum: Weight

### Compare previous version and version where offending jet is masked

In [6]:
filewithout = "/work/tharte/datasets/btag_efficiency_outputs/btag_eff_nobcuts_all_kl_proper_skimming_load_hist_last_only_m2p00_masking_offending_jet/output_all.coffea"
filewith = "/work/tharte/datasets/btag_efficiency_outputs/btag_eff_nobcuts_all_kl_proper_skimming_load_hist_last_only_m2p00/output_all.coffea"
histwithout = load(filewithout)["variables"]
histwith = load(filewith)["variables"]
print(histwith.keys())

dict_keys(['nJets', 'jet_eta_1', 'jet_pt_1', 'jet_phi_1', 'jet_eta_2', 'jet_pt_2', 'jet_phi_2', 'jet_eta_3', 'jet_pt_3', 'jet_phi_3', 'jet_eta_4', 'jet_pt_4', 'jet_phi_4', 'jet_eta_5', 'jet_pt_5', 'jet_phi_5', 'bjets_deepJet_L_pt_eta_flav', 'bjets_deepJet_M_pt_eta_flav', 'bjets_deepJet_T_pt_eta_flav', 'bjets_deepJet_XT_pt_eta_flav', 'bjets_deepJet_XXT_pt_eta_flav', 'bjets_particleNet_L_pt_eta_flav', 'bjets_particleNet_M_pt_eta_flav', 'bjets_particleNet_T_pt_eta_flav', 'bjets_particleNet_XT_pt_eta_flav', 'bjets_particleNet_XXT_pt_eta_flav', 'bjets_robustParticleTransformer_L_pt_eta_flav', 'bjets_robustParticleTransformer_M_pt_eta_flav', 'bjets_robustParticleTransformer_T_pt_eta_flav', 'bjets_robustParticleTransformer_XT_pt_eta_flav', 'bjets_robustParticleTransformer_XXT_pt_eta_flav', 'jets_pt_eta_flav'])


In [200]:
histwith == histwithout

False

In [7]:
pnet_T_with =histwith["bjets_particleNet_T_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
pnet_T_without =histwithout["bjets_particleNet_T_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
print(pnet_T_with)

pnet_tot_with =histwith["jets_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
pnet_tot_without =histwithout["jets_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']


print(pnet_T_without)
pnet_XT_with =histwith["bjets_particleNet_XT_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
pnet_XT_without =histwithout["bjets_particleNet_XT_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
# print(pnet_XT_with)
# print(pnet_XT_without)
vXT1 = pnet_XT_with.values(flow=True)
vXT2 = pnet_XT_without.values(flow=True)
vt1 = pnet_tot_with.values(flow=True)
vt2 = pnet_tot_without.values(flow=True)

v1 = pnet_T_with.values(flow=True)
v2 = pnet_T_without.values(flow=True)
diff_mask =v1 != v2
print(np.any(diff_mask))
print(np.sum(diff_mask))
indices = np.argwhere(diff_mask)
print(indices)

for idx in list(indices):
    idx_uo = list(idx)
    for i in range(5):
        if idx_uo[i] == pnet_T_with.axes[i].size:
            idx_uo[i]=bh.overflow
    i_cat, i_var, i_pt, i_eta, i_flav = idx_uo
    
    print("Difference at:")
    print("  cat      :", pnet_T_without.axes[0][i_cat])
    print("  variation:", pnet_T_without.axes[1][i_var])
    print("  pt bin   :", pnet_T_without.axes[2][i_pt])
    print("  eta bin  :", pnet_T_without.axes[3][i_eta])
    print("  flav bin :", pnet_T_without.axes[4][i_flav])
    print("  h1=", v1[tuple(idx)], " h2=", v2[tuple(idx)])

print("Same indices at XT wp")
print(f"With the offending jet: {vXT1[tuple(indices[0])]}")
print(f"Without the offending jet: {vXT2[tuple(indices[0])]}")

print("Efficiencies to be found in the correctionlib file:")
print(f"With the offending jet: {v1[tuple(indices[0])]/vt1[tuple(indices[0])]}")
print(f"Without the offending jet: {vXT2[tuple(indices[0])]/vt2[tuple(indices[0])]}")



Hist(
  StrCategory(['inclusive'], name='cat', label='Category'),
  StrCategory(['nominal', 'pileupDown', 'pileupUp'], name='variation', label='Variation'),
  Variable([20, 30, 50, 70, 100, 140, 200, 300, 600, 1000], name='BJetGood_particleNet_T.pt', label='BJetGood_particleNet_T pT'),
  Variable([0, 2.4, 2.5], name='BJetGood_particleNet_T.abseta', label='BJetGood_particleNet_T eta'),
  Variable([-0.5, 3.5, 4.5, 5.5], name='BJetGood_particleNet_T.hadronFlavour', label='BJetGood_particleNet_T flav'),
  storage=Weight()) # Sum: WeightedSum(value=2.50308, variance=8.3877e-06) (WeightedSum(value=2.50367, variance=8.38967e-06) with flow)
Hist(
  StrCategory(['inclusive'], name='cat', label='Category'),
  StrCategory(['nominal', 'pileupDown', 'pileupUp'], name='variation', label='Variation'),
  Variable([20, 30, 50, 70, 100, 140, 200, 300, 600, 1000], name='BJetGood_particleNet_T.pt', label='BJetGood_particleNet_T pT'),
  Variable([0, 2.4, 2.5], name='BJetGood_particleNet_T.abseta', label='B

In [8]:
pnet_T_with =histwith["bjets_particleNet_XXT_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
pnet_T_without =histwithout["bjets_particleNet_XXT_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']



v1 = pnet_T_with.values(flow=True)
v2 = pnet_T_without.values(flow=True)
diff_mask =v1 != v2
print(np.any(diff_mask))
print(np.sum(diff_mask))
indices = np.argwhere(diff_mask)
print(indices)

for idx in list(indices):
    idx_uo = list(idx)
    for i in range(5):
        if idx_uo[i] == pnet_T_with.axes[i].size:
            idx_uo[i]=bh.overflow
    i_cat, i_var, i_pt, i_eta, i_flav = idx_uo
    
    print("Difference at:")
    print("  cat      :", pnet_T_with.axes[0][i_cat])
    print("  variation:", pnet_T_with.axes[1][i_var])
    print("  pt bin   :", pnet_T_with.axes[2][i_pt])
    print("  eta bin  :", pnet_T_with.axes[3][i_eta])
    print("  flav bin :", pnet_T_with.axes[4][i_flav])
    print("  h1=", v1[tuple(idx)], " h2=", v2[tuple(idx)])




False
0
[]


### Doing the same, but with a 0pt edge in the histograms

In [9]:
filewithout = "/work/tharte/datasets/btag_efficiency_outputs/new_0_pt_edge_btag_eff_nobcuts_all_kl_proper_skimming_load_hist_last_only_m2p00_masking_offending_jet/output_all.coffea"
filewith = "/work/tharte/datasets/btag_efficiency_outputs/new_0_pt_edge_btag_eff_nobcuts_all_kl_proper_skimming_load_hist_last_only_m2p00/output_all.coffea"
histwithout = load(filewithout)["variables"]
histwith = load(filewith)["variables"]
print(histwith.keys())

dict_keys(['nJets', 'jet_eta_1', 'jet_pt_1', 'jet_phi_1', 'jet_eta_2', 'jet_pt_2', 'jet_phi_2', 'jet_eta_3', 'jet_pt_3', 'jet_phi_3', 'jet_eta_4', 'jet_pt_4', 'jet_phi_4', 'jet_eta_5', 'jet_pt_5', 'jet_phi_5', 'bjets_deepJet_L_pt_eta_flav', 'bjets_deepJet_M_pt_eta_flav', 'bjets_deepJet_T_pt_eta_flav', 'bjets_deepJet_XT_pt_eta_flav', 'bjets_deepJet_XXT_pt_eta_flav', 'bjets_particleNet_L_pt_eta_flav', 'bjets_particleNet_M_pt_eta_flav', 'bjets_particleNet_T_pt_eta_flav', 'bjets_particleNet_XT_pt_eta_flav', 'bjets_particleNet_XXT_pt_eta_flav', 'bjets_robustParticleTransformer_L_pt_eta_flav', 'bjets_robustParticleTransformer_M_pt_eta_flav', 'bjets_robustParticleTransformer_T_pt_eta_flav', 'bjets_robustParticleTransformer_XT_pt_eta_flav', 'bjets_robustParticleTransformer_XXT_pt_eta_flav', 'jets_pt_eta_flav'])


In [10]:
pnet_T_with =histwith["bjets_particleNet_T_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
pnet_T_without =histwithout["bjets_particleNet_T_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
pnet_tot_with =histwith["jets_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
pnet_tot_without =histwithout["jets_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']

print(pnet_T_with)
print(pnet_T_without)
pnet_XT_with =histwith["bjets_particleNet_XT_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
pnet_XT_without =histwithout["bjets_particleNet_XT_pt_eta_flav"]["GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_skimmed"]['GluGlutoHHto4B_spanet_kl-m2p00_kt-1p00_c2-0p00_2022_postEE']
# print(pnet_XT_with)
# print(pnet_XT_without)
vXT1 = pnet_XT_with.values(flow=True)
vXT2 = pnet_XT_without.values(flow=True)
vt1 = pnet_tot_with.values(flow=True)
vt2 = pnet_tot_without.values(flow=True)

v1 = pnet_T_with.values(flow=True)
v2 = pnet_T_without.values(flow=True)
diff_mask =v1 != v2
print(np.any(diff_mask))
print(np.sum(diff_mask))
indices = np.argwhere(diff_mask)
print(indices)



for idx in list(indices):
    idx_uo = list(idx)
    for i in range(5):
        if idx_uo[i] == pnet_T_with.axes[i].size:
            idx_uo[i]=bh.overflow
    i_cat, i_var, i_pt, i_eta, i_flav = idx_uo
    
    print("Difference at:")
    print("  cat      :", pnet_T_without.axes[0][i_cat])
    print("  variation:", pnet_T_without.axes[1][i_var])
    print("  pt bin   :", pnet_T_without.axes[2][i_pt])
    print("  eta bin  :", pnet_T_without.axes[3][i_eta])
    print("  flav bin :", pnet_T_without.axes[4][i_flav])
    print("  h1=", v1[tuple(idx)], " h2=", v2[tuple(idx)])

print("Same indices at XT wp")
print(f"With the offending jet: {vXT1[tuple(indices[0])]}")
print(f"Without the offending jet: {vXT2[tuple(indices[0])]}")

print("Efficiencies to be found in the correctionlib file:")
print(f"With the offending jet: {v1[tuple(indices[0])]/vt1[tuple(indices[0])]}")
print(f"Without the offending jet: {vXT2[tuple(indices[0])]/vt2[tuple(indices[0])]}")


Hist(
  StrCategory(['inclusive'], name='cat', label='Category'),
  StrCategory(['nominal', 'pileupDown', 'pileupUp'], name='variation', label='Variation'),
  Variable([0, 20, 30, 50, 70, 100, 140, 200, 300, 600, 1000], name='BJetGood_particleNet_T.pt', label='BJetGood_particleNet_T pT'),
  Variable([0, 2.4, 2.5], name='BJetGood_particleNet_T.abseta', label='BJetGood_particleNet_T eta'),
  Variable([-0.5, 3.5, 4.5, 5.5], name='BJetGood_particleNet_T.hadronFlavour', label='BJetGood_particleNet_T flav'),
  storage=Weight()) # Sum: WeightedSum(value=2.50363, variance=8.38956e-06) (WeightedSum(value=2.50367, variance=8.38967e-06) with flow)
Hist(
  StrCategory(['inclusive'], name='cat', label='Category'),
  StrCategory(['nominal', 'pileupDown', 'pileupUp'], name='variation', label='Variation'),
  Variable([0, 20, 30, 50, 70, 100, 140, 200, 300, 600, 1000], name='BJetGood_particleNet_T.pt', label='BJetGood_particleNet_T pT'),
  Variable([0, 2.4, 2.5], name='BJetGood_particleNet_T.abseta', l